<a href="https://colab.research.google.com/github/fummicc1/mnist_sample/blob/master/mnist_keras2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras==2.2.4 tensorflow==1.14.0

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [0]:
batch_size = 128
num_classes = 10
epochs = 12 # 訓練データを何回繰り返して学習させるのか
img_rows, img_cols = 28, 28

In [0]:
# 学習データとテストデータに分割
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [0]:
# backendがTensorflowとTheanoで配列のshapeが異なるために2パターン記述
if K.image_data_format() == 'channels_first':
  # 1次元配列に変換
  x_train = x_train.reshape(s_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)

else:
  # 1次元配列に変換
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)

In [0]:
# 入力データ[0, 1]の範囲に正規化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# 255で割ったものを新たに変数とする
x_train /= 255
x_test /= 255

print('x_train shape : ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape :  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# ラベルをバイナリベクトルとして扱う
# Kerasはレベルを数値ではなく、0or1を要素に持つベクトルで扱うため
"""
例えば、サンプルに対するターゲットが「5」の場合次のような形になります。
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
"""
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# CNNネットワークの構築
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# 損失関数,最適化関数,評価指標を指定してモデルをコンパイル
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),
             metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
# モデルの学習
model.fit(x_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 148s 2ms/step - loss: 0.2609 - acc: 0.9198 - val_loss: 0.0616 - val_acc: 0.9798
Epoch 2/12
60000/60000 [==============================] - 147s 2ms/step - loss: 0.0865 - acc: 0.9744 - val_loss: 0.0438 - val_acc: 0.9859
Epoch 3/12
60000/60000 [==============================] - 147s 2ms/step - loss: 0.0651 - acc: 0.9803 - val_loss: 0.0346 - val_acc: 0.9887
Epoch 4/12
60000/60000 [==============================] - 148s 2ms/step - loss: 0.0549 - acc: 0.9835 - val_loss: 0.0332 - val_acc: 0.9889
Epoch 5/12
60000/60000 [==============================] - 148s 2ms/step - loss: 0.0461 - acc: 0.9862 - val_loss: 0.0315 - val_acc: 0.9898
Epoch 6/12
60000/60000 [==============================] - 147s 2ms/step - loss: 0.0423 - acc: 0.9876 - val_loss: 0.0290 - val_acc: 0.9900
Epoch 7/12
60000/60000 [====

In [0]:
# モデルの評価
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03059587843184663
Test accuracy: 0.9914


In [0]:
!pip install -U coremltools

Requirement already up-to-date: coremltools in /usr/local/lib/python3.6/dist-packages (3.3)


In [0]:
model.save('mnist_mlp_weights.h5')

In [0]:
import coremltools

coreml_model = coremltools.converters.keras.convert('mnist_mlp_weights.h5')
coreml_model.save('my_keras_model.mlmodel')

0 : conv2d_1_input, <keras.engine.input_layer.InputLayer object at 0x7f2b8ca54400>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x7f2b8ca543c8>
2 : conv2d_1__activation__, <keras.layers.core.Activation object at 0x7f2ba31e37b8>
3 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x7f2b8ca54588>
4 : conv2d_2__activation__, <keras.layers.core.Activation object at 0x7f2b8c8e56a0>
5 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0x7f2b8ca54630>
6 : flatten_1, <keras.layers.core.Flatten object at 0x7f2b8ca03ac8>
7 : dense_1, <keras.layers.core.Dense object at 0x7f2b8c9e6828>
8 : dense_1__activation__, <keras.layers.core.Activation object at 0x7f2b8c8e5898>
9 : dense_2, <keras.layers.core.Dense object at 0x7f2b8c963438>
10 : dense_2__activation__, <keras.layers.core.Activation object at 0x7f2b8c8a0898>


Recommendation: This model has at least one multiarray input/output of type double.
For large sized arrays, multiarrays of type float32 are more efficie

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)